In [1]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
from pyomo.environ import *
import numpy as np

     |████████████████████████████████| 9.1 MB 18.3 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148486 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to un

In [76]:
model=ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
d={4:80,6:50,7:100}
a=np.array(([[2,0,0],[0,2,0],[0,0,2]]))
a.shape
pattern_list=range(a.shape[1])
model.lamb=Var(pattern_list,domain=NonNegativeIntegers)
model.pattern_count=Objective(expr=sum(model.lamb[i] for i in pattern_list))
model.c=ConstraintList()
for j in d:
  k=demand_index[j]
  #print()
  model.c.add(sum(model.lamb[k]*a[pattern,k] for pattern in pattern_list)>=d[j])



In [77]:
demand_index={4:0,6:1,7:2}


4 0
6 1
7 2


AttributeError: ignored

In [78]:
SolverFactory('cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 115.0
  Upper bound: 115.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Number of binary variables: 0
  Number of integer variables: 3
  Number of nonzeros: 0
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
    

In [79]:
# display all duals
dual_vector=[]
print ("Duals")
for c in model.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    for index in c:
        print ("      ", index, model.dual[c[index]])
        dual_vector.append(model.dual[c[index]])
print(dual_vector)

Duals
   Constraint c
       1 0.0
       2 0.0
       3 0.0
[0.0, 0.0, 0.0]


In [40]:
model.dual.display()

dual : Direction=Suffix.IMPORT_EXPORT, Datatype=Suffix.FLOAT
    Key  : Value
    c[1] :   0.0
    c[2] :   0.0
    c[3] :   0.0


In [80]:
pricing=ConcreteModel()
#model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
#d={4:80,6:50,7:100}
#a=np.array(([[3,0,0],[0,2,0],[0,0,2]]))
#a.shape
#pattern_list=range(a.shape[1])
#model.lamb=Var(pattern_list,domain=NonNegativeIntegers)
lenght=[4,6,7]
m=len(dual_vector)
pricing.x=Var(range(m),domain=NonNegativeIntegers)
pricing.least_red_cost=Objective(expr=(1-sum(dual_vector[i]*pricing.x[i] for i in [0,1,2])))
pricing.c=ConstraintList()
pricing.c.add(sum(lenght[i]*pricing.x[i] for i in range(m))<=15)
SolverFactory('cbc').solve(pricing).write()

    solver failure.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1.0
  Upper bound: 1.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Number of binary variables: 0
  Number of integer variables: 3
  Number of nonzeros: 0
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch 

In [84]:
if(pricing.least_red_cost.expr<0):
  print("Improvements Exists")
  print("Add x to a")
  print("x is",pricing.x.value)
else:
  print("No Improvements")

No Improvements
